In [ ]:
"""
Map each description to vector using word2vec.
"""

import os
import csv
import random
import gensim
import numpy as np

word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
print("Loaded word vectors successfully!")

def doc_to_vec(words, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in words if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

In [18]:
import pickle
import os.path
from sklearn.decomposition import PCA

train_description_feature_map = pickle.load(open('./../features/12_4_[train]_description_feature_map_py27.pkl', 'rb'))
test_derscription_feature_map = pickle.load(open('./../features/12_4_[test]_description_feature_map_py27.pkl', 'rb'))
print "loaded description map..."

loaded description map...


In [3]:
train_img_map = pickle.load(open('./../features/12_1_train_img_feature_map_py27.pkl', 'rb'))
test_img_map = pickle.load(open('./../features/12_1_test_img_feature_map_py27.pkl', 'rb'))
print "loaded img feature map..."

train_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], train_img_map)))
test_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], test_img_map)))

loaded description map...
loaded img feature map...


In [ ]:
print test_derscription_feature_map[0]

In [22]:
train_doc_vecs = np.asarray(list(map(lambda x: x["doc_vec"], train_description_feature_map)))
test_doc_vecs = np.asarray(list(map(lambda x: x["doc_vec"], test_derscription_feature_map)))

In [20]:
"""
PLSR
"""
from sklearn.cross_decomposition import PLSRegression
model_name = "./../models/pls_800_pool5_to_word2vec_n_v_adj_300.pkl"
if os.path.exists(model_name):
    model = pickle.load(open(model_name, 'rb'))
else:
    # train PLSR.
    model = PLSRegression(n_components=800)
    model.fit(train_pool5_img, train_doc_vecs)
    pickle.dump(model, open(model_name, 'wb'), protocol=2)
print "model loaded..."

model loaded...


/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator PLSRegression from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [24]:
from scipy.spatial.distance import cdist
import csv

pred_test_pool5 = model.predict(test_pool5_img)
dist = cdist(test_doc_vecs, pred_test_pool5, 'cosine') # CHANGE!!


print("description * images dist matrix, shape:", dist.shape)
sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.

with open('./../submission/submission_plsr_800_pool5_to_word2vec_n_v_ajd_300.csv', 'w') as csvfile:
        # write csv header
        fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for i, row in enumerate(sorted_id):
            top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
            res = {}
            res['Descritpion_ID'] = str(i) + ".txt" # file name
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

print("Writing Complete.")

('description * images dist matrix, shape:', (2000, 2000))
Writing Complete.
